<a href="https://colab.research.google.com/github/blue0620/aozorasummary/blob/main/get_summary_from_cohere_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#command-R-plusから青空文庫の要約文を取得してgoogle driveに保存するスクリプト


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install cohere --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.9/252.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 49.3 MB/s eta 0:00:00


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 whi

In [ ]:
outputpath="/content/drive/MyDrive/commandrplus_aozorasummary/"
basequery="次の文章を簡潔に要約して。（200token以内）"

In [ ]:
import cohere

co = cohere.ClientV2("xxx")#自分のAPIトークンに書き換える

In [ ]:
import pandas as pd
from datasets import load_dataset
ds = load_dataset('globis-university/aozorabunko-clean')
#ds = ds.filter(lambda row: row['meta']['姓'] == '宮沢')
#ds = ds.filter(lambda row: row['meta']['名'] == '賢治')
df = pd.DataFrame(ds["train"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.96k [00:00<?, ?B/s]

aozorabunko-dedupe-clean.jsonl.gz:   0%|          | 0.00/241M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16951 [00:00<?, ? examples/s]

In [ ]:
import os
import time
for index,row in df.iterrows():
    print(row)
    id=row['meta']['作品ID']
    if os.path.exists(os.path.join(outputpath,id+'.txt')):
        continue
    query=basequery+row['text'][:10000]
    response = co.chat(
        model="command-r-plus-08-2024",
        messages=[{"role": "user", "content": query}],
    )
    res=response.message.content[0].text
    print(res)
    with open(os.path.join(outputpath,id+'.txt'), 'w') as f:
        f.write(res)
    time.sleep(3)